In [18]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import xgboost as xgb

In [19]:
xtrain = np.genfromtxt('X_train.csv', delimiter=',', skip_header=1)
ytrain = np.genfromtxt('y_train.csv', delimiter=',', skip_header=1)

xtrain = xtrain[:, 1:]
ytrain = ytrain[:, 1:]
nb_train = xtrain.shape[0] #number of data points in xtrain

xtrain = pd.DataFrame(xtrain)
xtrain = xtrain.fillna(xtrain.median())
xtrain = np.array(xtrain)

In [20]:
#random shuffling of data 

perm = np.random.permutation(xtrain.shape[0])
xtrain = xtrain[perm, :]
ytrain = xtrain[perm, :]

In [21]:
#splitting data for CV 

nb_folds = 10
parts_x = np.array_split(xtrain, nb_folds, axis=0)
parts_y = np.array_split(ytrain, nb_folds, axis=0)
 

In [22]:
avgr2_train = 0
avgr2_val = 0
for cv_iter in range(0, nb_folds):
    curr_train_x = np.vstack([p for i,p in enumerate(parts_x) if i != cv_iter])
    curr_train_y = np.vstack([p for i,p in enumerate(parts_y) if i != cv_iter]).ravel()
 
    curr_val_x = parts_x[cv_iter]
    curr_val_y = parts_y[cv_iter].ravel()
 
    model = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)
 
    model.fit(curr_train_x, curr_train_y.ravel())
    train_prediction = model.predict(curr_train_x)
    r2_train = r2_score(train_prediction, curr_train_y)
    avgr2_train += r2_train
 
    val_prediction = model.predict(curr_val_x)
    r2_val = r2_score(val_prediction, curr_val_y)
    avgr2_val += r2_val
    print('[Fold {:2d}] train: {:5.2f} val:{:5.2f}'.format(cv_iter, r2_train, r2_val))
   
avgr2_train /= nb_folds
avgr2_val /= nb_folds
print('[Average] train: {:5.2f} val: {:5.2f}'.format(avgr2_train, avgr2_val))


XGBoostError: [00:52:39] C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:65: Check failed: preds.Size() == info.labels_.Size() (1090 vs. 906880) : labels are not correctly providedpreds.size=1090, label.size=906880

In [50]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

In [51]:
# Step 0: Load the proper csvs

xtrain = pd.read_csv("X_train.csv")
xtrain = xtrain.iloc[:, 1:]
xtest = pd.read_csv("X_test.csv")
xtest = xtest.iloc[:, 1:]
ytrain = pd.read_csv("Y_train.csv")

In [52]:
# Step 1: Treat missing values
# Treat missing values as column medians. Important (apparently) to use the medians from the training set in the test set

xtrain = xtrain.fillna(xtrain.median())
xtest = xtest.fillna(xtrain.median())

In [53]:
# Step 2: Scale the training and test data

scaler = preprocessing.StandardScaler()

xtrain_scaled = scaler.fit_transform(xtrain)
xtrain = pd.DataFrame(xtrain_scaled, columns = xtrain.columns)
xtest_scaled = scaler.fit_transform(xtest)
xtest = pd.DataFrame(xtest_scaled, columns = xtest.columns)

In [54]:
# Step 3: Remove unnecessary id column from ytrain that just ***** things up

ytrain1 = ytrain.loc[:, "y"]
ytrain2 = pd.DataFrame(data = ytrain1.values, columns= ['y'])
ytrain = ytrain2

In [55]:


# Do weird Andreas PCA thing

from sklearn import decomposition
import matplotlib.pyplot as plt
pca = decomposition.PCA(n_components=2)
principal_components = pca.fit_transform(xtrain)
xpca = pd.DataFrame(data = principal_components, columns = ['PC1', 'PC2'])
# Do weird Andreas PCA thing

from sklearn import decomposition
import matplotlib.pyplot as plt
pca = decomposition.PCA(n_components=2)
principal_components = pca.fit_transform(xtrain)
xpca = pd.DataFrame(data = principal_components, columns = ['PC1', 'PC2'])

outliers = (xpca['PC1'] <= 9) & (xpca['PC1'] >= -8) & (xpca['PC2'] <= 7) & (xpca['PC2'] >= -7)
print(np.count_nonzero(outliers))
xtrain = xtrain[outliers]
ytrain = ytrain[outliers]


741


In [56]:
# k-fold cross validation evaluation of xgboost model

from numpy import loadtxt
import xgboost
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from sklearn import linear_model

k = 10

cv_means = []
cv_stds = []
model = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)
scores = cross_val_score(estimator = model,
                             X = xtrain,
                             y = ytrain,
                             scoring = 'r2',
                             cv = k)
print(np.mean(scores))
#cv_means.append(np.mean(scores))
#cv_stds.append(np.std(scores))

0.5490635103006762


In [33]:
model.fit(xtrain,ytrain)
predict = model.predict(xtest)
index = pd.read_csv("sample.csv")
index['y'] = predict

index.to_csv("trial.csv")

Need to try some fetures selection and outlier detection shit

In [35]:
np.mean(scores)

0.5175284544897842